In [124]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import itertools
import os
import time
import datetime as dt 
from pandas_datareader import data as web
import yfinance as yf
from scipy.stats import zscore

from functions.datareader import pull_stock_data

<h1 style="color:orange">Load data using YFinance and save to local</h1>

In [39]:
# source for yfinance: https://pypi.org/project/yfinance/

In [40]:
start = dt.date(2021, 1, 1)
end = dt.date(2022, 1, 1)

# meta = yf.Tickers(['BBL.BK', 'KBANK.BK'])

In [41]:
# price_df = meta.history(period = 'MAX').sort_index()

In [42]:
# price_df[[c for c in price_df.columns if c[1] == 'BBL.BK']].tail(20)

In [43]:
sectors = {
    'banking': ['BBL', 'KBANK', 'SCB', 'BAY', 'TISCO', 'KTB', 'KKP', 'TTB'],
    'hospital': ['BDMS', 'BCH', 'BH', 'THG', 'PR9', 'EKH', 'IMH'],
    'infrastructure': ['AOT', 'BAFS', 'BTS', 'BEM', 'DMT'],
    'consumer_staple': ['BJC', 'OR', 'CPALL', 'CRC', 'MAJOR', 'GFPT', 'CPF', 'MAKRO', 'M'],
    'consumer_discretionary': ['COM7', 'CPW', 'SYNEX'],
    'technology': ['BBIK', 'IIG', 'BE8'],
    'industrial': ['WHA', 'AMATA'],
    'material': ['SCGP', 'HMPRO', 'GLOBAL', 'DOHOME', 'TPIPL', 'STGT', 'THMUI'],
    'utilities': ['RATCH', 'BGRIM', 'GULF', 'TPIPP', 'EGCO', 'EA', 'BANPU', 'ACE'],
    'petrochemistry': ['PTTGC', 'BCP', 'IRPC', 'IVL'],
    'real_estate': ['SIRI', 'QH', 'AP', 'SPALI', 'ORI', 'LALIN'],
    'hotel': ['MINT', 'CENTEL', 'ERW', 'AWC', 'SPA'],
    'consumer_finance': ['SAWAD', 'KTC', 'AEONTS', 'TIDLOR', 'MTC'],
    'insurance': ['BLA']
}

all_tickers = sectors.values()
all_tickers = [v + '.BK' for s in all_tickers for v in s]

In [44]:
all_meta = yf.Tickers(all_tickers)

In [45]:
all_price_df = all_meta.history(period = 'max')

[*********************100%***********************]  73 of 73 completed


In [46]:
all_price_df.tail(1)

Close                                                          \
           ACE.BK AEONTS.BK AMATA.BK AOT.BK AP.BK AWC.BK BAFS.BK BANPU.BK   
Date                                                                        
2023-02-23   2.46     205.0     20.1   72.5  12.5   6.05   33.25     11.3   

                           ...     Volume                                \
           BAY.BK BBIK.BK  ...    STGT.BK  SYNEX.BK     THG.BK THMUI.BK   
Date                       ...                                            
2023-02-23  29.75   121.0  ...  8847887.0  838052.0  1571106.0  97511.0   

                                                                      \
            TIDLOR.BK   TISCO.BK    TPIPL.BK   TPIPP.BK       TTB.BK   
Date                                                                   
2023-02-23  4892257.0  3598604.0  29016711.0  1105594.0  184096615.0   

                        
                WHA.BK  
Date                    
2023-02-23  52493989.0  

[1 rows x 511 columns]

In [47]:
# Price
parent_dir = './data/set'
for t in all_tickers:
    t_trim = t.replace('.BK', '')
    ticker_dir = f'{parent_dir}/{t_trim}'
    if not os.path.exists(ticker_dir):
        os.mkdir(ticker_dir)
    
    ticker_cols = [c for c in all_price_df.columns if c[1] == t]
    ticker_df = all_price_df[ticker_cols].dropna(axis = 0)
    ticker_df.columns = [c[0].lower() for c in ticker_df.columns]
    ticker_df.insert(0, 'ticker', t_trim)
    ticker_df.index.name = 'date'

    price_dir = f'{ticker_dir}/price'
    if not os.path.exists(price_dir):
        os.mkdir(price_dir)
    years = sorted(list(set(ticker_df.index.year)))
    for y in years:
        year_df = ticker_df[ticker_df.index.year == y]
        year_df.to_parquet(f'{price_dir}/{str(y)}.parquet')

In [2]:
# # to plot
# year_start = 2016
# # tickers = {'BBL': 'blue', 'KBANK': 'green', 'KKP': 'purple'}
# tickers = {'BGRIM': 'green', 'GULF': 'red'}
# plt.figure(figsize = (10, 3))
# for t in tickers:
#     ticker_df = all_df[all_df['ticker'] == t]['Close']

#     # ? filter year
#     ticker_df = ticker_df[ticker_df.index.year >= year_start]

#     first_value = ticker_df.iloc[0]
#     # ? normalize prices
#     ticker_df = ticker_df.div(first_value)
#     ticker_df.plot(color = tickers[t], label = t)
# plt.legend(loc = 'upper right')

<h1 style="color:salmon">Load data from local and use only close price</h1>

In [48]:
raw_df = pd.read_parquet('./data/set')
raw_df.tail()

,ticker,close,dividends,high,low,open,stock splits,volume
date,,,,,,,,
2023-02-17,WHA,3.86,0.0,3.86,3.82,3.84,0.0,30699900.0
2023-02-20,WHA,3.86,0.0,3.88,3.84,3.86,0.0,15987800.0
2023-02-21,WHA,4.00,0.0,4.00,3.90,3.90,0.0,326198100.0
2023-02-22,WHA,3.94,0.0,4.00,3.92,3.96,0.0,136125400.0
2023-02-23,WHA,3.94,0.0,3.96,3.90,3.94,0.0,52493989.0


In [60]:
# map sector using the sector dict above
vk = [(k, v) for k, v in sectors.items()]
vk[0]
sector_mapper = {sub: s[0]for s in vk for sub in s[1]}
# sector_mapper

raw_df['sector'] = raw_df['ticker'].map(sector_mapper)


<p>We'll define a starting period of our analysis. Therefore we'll need to know whether there are stocks of which the daily price data doesn't exist yet in that period.</p>

In [61]:
start_date = dt.date(2015, 1, 1)

first_date_df = raw_df[['ticker']].reset_index().groupby('ticker').min()
first_date_df['is_available_since_start'] = first_date_df['date'].apply(lambda x: True if x.date() <= start_date else False)
# first_date_df.head()

available_tickers = first_date_df[first_date_df['is_available_since_start'] == True].reset_index()['ticker']

In [62]:
# filter only selected stocks
df = raw_df.reset_index().merge(available_tickers, left_on = 'ticker', right_on = 'ticker').set_index('date')
df.head()

,ticker,close,dividends,high,low,open,stock splits,volume,sector
date,,,,,,,,,
2001-12-11,AEONTS,4.823158,0.0,5.126979,4.405404,4.481359,0.0,78469000.0,consumer_finance
2001-12-12,AEONTS,4.709227,0.0,5.013048,4.709227,4.937093,0.0,31076500.0,consumer_finance
2001-12-13,AEONTS,4.557315,0.0,4.785181,4.557315,4.747203,0.0,11258000.0,consumer_finance
2001-12-14,AEONTS,4.823158,0.0,4.899113,4.443382,4.557315,0.0,19788500.0,consumer_finance
2001-12-17,AEONTS,4.823158,0.0,4.899113,4.823158,4.899113,0.0,3423000.0,consumer_finance


In [67]:
close_df = df[['ticker', 'sector', 'close']]
close_df.head()

,ticker,sector,close
date,,,
2001-12-11,AEONTS,consumer_finance,4.823158
2001-12-12,AEONTS,consumer_finance,4.709227
2001-12-13,AEONTS,consumer_finance,4.557315
2001-12-14,AEONTS,consumer_finance,4.823158
2001-12-17,AEONTS,consumer_finance,4.823158


<h2 style="color:cream">Create Factors</h2>

In [129]:
# 1 year momemtum (1 year return)
price_df = close_df.reset_index().pivot(index = 'date', columns = ['sector', 'ticker'], values = 'close') \
                .sort_index()

return_1yr = price_df.pct_change(periods = 252).dropna(axis = 0)

# demean by sector
momemtum_demean = None

for s in sectors:
    sector_return = return_1yr[[c for c in return_1yr.columns if c[0] == s]]
    sector_avg = sector_return.mean(axis = 1)
    sector_demean = sector_return - np.broadcast_to(sector_avg.values.reshape(-1, 1), sector_return.shape)
    momemtum_demean = sector_return if momemtum_demean is None else momemtum_demean.merge(sector_return, left_index = True, right_index = True)

# rank 
momentum_rank = momemtum_demean.rank(axis = 1, method = 'max')
momentum_rank.head()

# z-score
momentum_factor_df = momentum_rank.apply(zscore, axis = 1)

In [ ]:
# Mean reversion factor
N_DATE = 5
SMOOTH = False